In [6]:
import pandas as pd
import numpy as np
import nltk
df = pd.read_csv('../data/press_all.csv')

## Pre processing

In [11]:
# Replace nan with ""
df = df.fillna("")
df

,id,siren,url_article,title,text
0,0,572067684,http://www.autoactu.com/?page=2&id_actu=24240&...,autoactu.com - l'information automobile profes...,Bosch recrute des diplômés hautement qualifi...
1,1,790503973,http://www.ouest-france.fr/bretagne/lannion-22...,You have been blocked,Please enable JS and disable any ad blocker
2,2,433977980,http://www.businesswire.fr/news/fr/20160308006...,Axway s'associe à Navitas pour accompagner la ...,
3,3,495232373,http://www.next-finance.net/Sofimac-Partners-a...,Marchés Privés - Sofimac Partners apporte 450 ...,Présent au capital de cette jeune entreprise ...
4,4,572050169,http://rss.usinenouvelle.com/~r/a-la-une/~3/Gs...,"Les finances dans le vert, Latécoère veut mode...",\n\n\n\n \n\nIdentifiez-vous\n L'Usine Aéro P...
...,...,...,...,...,...
76509,76509,510050461,https://www.lejsl.com/edition-bresse/2019/10/1...,Sornay | Un salarié prend la succession de Lec...,JOURNAL DE SAÔNE ET LOIRE Conditions généra...
76510,76510,445720717,https://abonne.lunion.fr/id102153/article/2019...,,
76511,76511,480012277,https://www.lemondedudroit.fr/deals/66470-jean...,Jeantet conseille Formel D - LE MONDE DU DROIT...,UTILISATION DES COOKIES : en poursuivant votr...
76512,76512,414819409,http://www.leparisien.fr/val-de-marne-94/ile-d...,"Ile-de-France : à Villeneuve-Saint-Georges, le...","« S'ils veulent nous virer, qu'ils le fassent..."


In [12]:
#Delete all articles with length 0 
# Get names of articles with length 0
indexNames = df[ (df['text']) == "" ].index

# Delete these row indexes from dataFrame
df.drop(indexNames , inplace=True)
df

,id,siren,url_article,title,text
0,0,572067684,http://www.autoactu.com/?page=2&id_actu=24240&...,autoactu.com - l'information automobile profes...,Bosch recrute des diplômés hautement qualifi...
1,1,790503973,http://www.ouest-france.fr/bretagne/lannion-22...,You have been blocked,Please enable JS and disable any ad blocker
3,3,495232373,http://www.next-finance.net/Sofimac-Partners-a...,Marchés Privés - Sofimac Partners apporte 450 ...,Présent au capital de cette jeune entreprise ...
4,4,572050169,http://rss.usinenouvelle.com/~r/a-la-une/~3/Gs...,"Les finances dans le vert, Latécoère veut mode...",\n\n\n\n \n\nIdentifiez-vous\n L'Usine Aéro P...
5,5,794458075,http://feedproxy.google.com/~r/TracesEcrites/~...,La jeune pousse In’Air Solutions traque les po...,MENU Bourgogne-Franche-Comté Grand-Est - ...
...,...,...,...,...,...
76508,76508,493255848,http://www.actu-transport-logistique.fr//dossi...,,Transport Supply Chain Magazine | Dossier...
76509,76509,510050461,https://www.lejsl.com/edition-bresse/2019/10/1...,Sornay | Un salarié prend la succession de Lec...,JOURNAL DE SAÔNE ET LOIRE Conditions généra...
76511,76511,480012277,https://www.lemondedudroit.fr/deals/66470-jean...,Jeantet conseille Formel D - LE MONDE DU DROIT...,UTILISATION DES COOKIES : en poursuivant votr...
76512,76512,414819409,http://www.leparisien.fr/val-de-marne-94/ile-d...,"Ile-de-France : à Villeneuve-Saint-Georges, le...","« S'ils veulent nous virer, qu'ils le fassent..."


In [13]:
import re
import contractions
import string
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize 
from nltk.stem import PorterStemmer
from nltk.stem.snowball import FrenchStemmer
import spacy
from spacy_lefff import LefffLemmatizer, POSTagger

nlp = spacy.load('fr')
nlp.max_length = 68111459
french_lemmatizer = LefffLemmatizer()
nlp.add_pipe(french_lemmatizer, name='lefff')
translator = str.maketrans('', '', string.punctuation)


2020-03-31 06:17:10,424 - spacy_lefff.lefff - INFO - New LefffLemmatizer instantiated.
2020-03-31 06:17:10,426 - spacy_lefff.lefff - INFO - Reading lefff data...
2020-03-31 06:17:10,908 - spacy_lefff.lefff - INFO - Successfully loaded lefff lemmatizer


In [57]:
translator = str.maketrans('', '', string.punctuation)

# Tokenize text
def preprocessing(doc):
        result = []
        doc = doc.replace('\n', ' ')
        # To lowercase 
        doc = doc.lower()
        # Remove numbers
        doc = re.sub("\d+", "", doc)
        doc = doc.replace('m€', '')
        doc = doc.replace('k€', '')    
        sentences = sent_tokenize(doc)
        # Lemmentization (same as stemming but based on knowledge base)
        for sent in sentences: 
            # Delete punctuation
            sent = sent.translate(translator)
            lemmentization = []
            doc_ = nlp(sent,disable = ['ner', 'parser'])
            for d in doc_:
                lemmentization.append(d.lemma_)
            lemmentization.append('\n')
            res_ = " ".join(lemmentization)
            result.append(res_)
        return result 

def test(dataframe,index=0):
    f = open('data' + str(index) + '.txt' , 'w')
    for i,row in dataframe.iterrows():
        result = "".join(preprocessing((row['text'])))
        f.write(result)  # python will convert \n to os.linesep
    f.close()  

In [58]:
%%time 
result = " ".join(preprocessing((df.text[0])))

CPU times: user 24.7 ms, sys: 0 ns, total: 24.7 ms
Wall time: 24.1 ms


In [59]:
20*60000/1000/60

20.0

In [61]:
%%time
x = [" ".join(preprocessing(df.text[0])),"".join(preprocessing(df.text[1])),"".join(preprocessing(df.text[3]))]
result = " ".join(preprocessing((df.text[0])))
result

f = open('data' + str(0) + '.txt' , 'w')
for i in x:
    f.write(i)  
f.close()

CPU times: user 80.9 ms, sys: 121 µs, total: 81.1 ms
Wall time: 80.5 ms


In [ ]:
%%time
import multiprocessing
chunks = np.array_split(df,4)
manager = multiprocessing.Manager()
threads = []
for index,chunk in enumerate(chunks):
    thread = multiprocessing.Process(target=test, args=(chunk,index))
    thread.start()
    threads.append(thread)
for thread in threads:
    thread.join()


In [ ]:
! cat data*.txt > merge

In [ ]:
# Check average length of article 
df['length'] = df.text.apply(lambda x: len(x))
max_ = df.length.max()
print("Max: {}".format(max_))
min_ = df.length.min()
print("Min: {}".format(min_))
mean = df.length.mean()
print("Mean: {}".format(mean))


In [14]:
f = open('data.txt', 'w')
for i,row in df.iterrows():
    f.write(row['text'] + '\n')  # python will convert \n to os.linesep
f.close()  

In [1]:
import fasttext
model = fasttext.train_unsupervised('merge',thread=4)

In [6]:
import pandas as pd
df_mapping = pd.read_csv('../data/siren_name_map.csv')

In [7]:
from collections import defaultdict
mapping = defaultdict(str)
for i,row in df_mapping.iterrows():
    mapping[row.siren] = row.legal_name

In [ ]:
df_nearest = pd.DataFrame(columns=['siren', 'legal_name', 'nearest_neighbours'])
# Append rows in Empty Dataframe by adding dictionaries
#dfObj = dfObj
for siren,name in mapping.items():
    df_nearest.append({'siren': siren, 'legal_name': name, 'nearest_neighbours': model.get_nearest_neighbors(name,5)}, ignore_index=True)

In [ ]:
df_nearest

In [1]:
import fasttext as ft
model = ft.load_model('../data/model_unprocessed24-03-2020.bin')

In [2]:
model.get_nearest_neighbors('ipsen,5')

[(0.818486750125885, "'ipsen',"),
 (0.7570090293884277, "'spindler',"),
 (0.7473209500312805, "'medincell',"),
 (0.7468072772026062, "'genzyme',"),
 (0.7457915544509888, "'deinobiotics',"),
 (0.7412319183349609, "'anti-inflammatoire',"),
 (0.7395758628845215, "'lutathera',"),
 (0.7371309399604797, "'neurone',"),
 (0.7369047403335571, "'herment',"),
 (0.7367886304855347, "'neuroendocrine',")]

In [ ]:
from collections import defaultdict
nearest = defaultdict(str)
for x in model.get_words():
    nearest[x] = model.get_nearest_neighbors(x)